In [8]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import re
import yfinance as yf
import numpy as np
from datetime import datetime
import time
from scipy.stats import norm


In [2]:
def get_todays_price(ticker_symbol):
    """
    Fetches the most recent trading price for the given ticker.
    """
    ticker = yf.Ticker(ticker_symbol)
    try:
        data = ticker.history(period="1d", interval="1m")
        if data.empty:
            raise ValueError("No intraday data returned.")
        price = data["Close"].iloc[-1]
        return round(price, 2)
    except Exception as e:
        print(f"Error fetching price for {ticker_symbol}: {e}")
        return None

In [3]:
def fetch_full_option_chain(ticker_symbol, max_expirations=5, sleep_time=1):
    """
    Fetches all options (calls & puts) for a ticker across multiple expiration dates.
    """
    ticker = yf.Ticker(ticker_symbol)
    expirations = ticker.options[:max_expirations]
    all_options = []

    for expiry in expirations:
        try:
            chain = ticker.option_chain(expiry)
            calls = chain.calls.copy()
            puts = chain.puts.copy()

            # Label type
            calls["type"] = "call"
            puts["type"] = "put"

            # Add expiration column
            calls["expiration"] = expiry
            puts["expiration"] = expiry

            # Merge calls & puts
            combined = pd.concat([calls, puts], ignore_index=True)
            all_options.append(combined)

            print(f"Fetched {len(combined)} contracts for expiry {expiry}")
            time.sleep(sleep_time)

        except Exception as e:
            print(f"⚠️ Failed on {expiry}: {e}")
            continue

    return pd.concat(all_options, ignore_index=True) if all_options else pd.DataFrame()

In [4]:
def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    if T <= 0 or sigma <= 0:
        return max(0, S - K) if option_type == 'call' else max(0, K - S)
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

In [5]:
def calculate_bs_prices(df, S, r=0.045):
    df = df.copy()
    df["T"] = df["days_to_expiration"] / 365

    df["bs_price"] = df.apply(
        lambda row: black_scholes_price(
            S=S,
            K=row["strike"],
            T=row["T"],
            r=r,
            sigma=row["impliedVolatility"],
            option_type=row["type"]
        ),
        axis=1
    )

    df["pricing_error"] = df["lastPrice"] - df["bs_price"]
    return df

In [7]:
#df.to_csv("AAPL_option_chain.csv", index=False)

In [6]:
def enrich_option_data(df, underlying_price):
    """
    Enriches options DataFrame with moneyness & days-to-expiration.
    """
    df = df.copy()
    df["moneyness"] = df["strike"] / underlying_price
    df["current_date"] = pd.Timestamp.today().normalize()
    df["expiration"] = pd.to_datetime(df["expiration"])
    df["days_to_expiration"] = (df["expiration"] - df["current_date"]).dt.days
    df.drop(columns=["current_date"], inplace=True)
    return df

In [9]:
"""# 🚀 **Run the Full Pipeline**
ticker_symbol = "AAPL"

# Step 1: Fetch latest stock price
S = get_todays_price(ticker_symbol)
print(f"✅ Latest {ticker_symbol} price: ${S}")

# Step 2: Fetch option chain
df = fetch_full_option_chain(ticker_symbol, max_expirations=100)

# Step 3: Enrich the data
enriched_df = enrich_option_data(df, underlying_price=S)

# Display result
print(enriched_df.head())
"
"""


'# 🚀 **Run the Full Pipeline**\nticker_symbol = "AAPL"\n\n# Step 1: Fetch latest stock price\nS = get_todays_price(ticker_symbol)\nprint(f"✅ Latest {ticker_symbol} price: ${S}")\n\n# Step 2: Fetch option chain\ndf = fetch_full_option_chain(ticker_symbol, max_expirations=100)\n\n# Step 3: Enrich the data\nenriched_df = enrich_option_data(df, underlying_price=S)\n\n# Display result\nprint(enriched_df.head())\n"\n'

In [10]:
#enriched_df.to_csv("AAPL_option_chain.csv", index=False)

In [9]:
# Choose your ticker
ticker_symbol = "AAPL"

# Step 1: Get underlying price
S = get_todays_price(ticker_symbol)

# Step 2: Fetch option chain
df = fetch_full_option_chain(ticker_symbol, max_expirations=5)

# Step 3: Enrich with derived metrics
enriched_df = enrich_option_data(df, underlying_price=S)

# Step 4: Apply Black-Scholes pricing
bs_df = calculate_bs_prices(enriched_df, S)

# Step 5: View results
bs_df[["contractSymbol", "type", "strike", "lastPrice", "bs_price", "pricing_error"]].head()


Fetched 105 contracts for expiry 2025-03-14
Fetched 164 contracts for expiry 2025-03-21
Fetched 76 contracts for expiry 2025-03-28
Fetched 73 contracts for expiry 2025-04-04
Fetched 52 contracts for expiry 2025-04-11


,contractSymbol,type,strike,lastPrice,bs_price,pricing_error
0,AAPL250314C00100000,call,100.0,133.96,127.941619,6.018381
1,AAPL250314C00120000,call,120.0,120.95,107.901410,13.048590
2,AAPL250314C00130000,call,130.0,96.75,97.925713,-1.175713
3,AAPL250314C00140000,call,140.0,88.89,88.009557,0.880443
4,AAPL250314C00145000,call,145.0,84.85,82.925437,1.924563
